# Geo-py User Guide

## Introduction:

The purpose of this Python library is to enable geotechnical engineers to perform routine design calculations quickly, effortlessly, and accurately. Many of the more complicated design procedures for geotechnical engineers are already available as commercial software, but simpler procedures tend to be left to individual firms to create their own Excel templates, or to be done by hand. Excel is an adequate tool for some procedures, but other procedures are better suited for a more robust programming language due to the amount of logic and branching paths a design can take. With Geo-py, engineers can perform routine analyses and produce calculation reports effortlessly, leaving more time to focus on the big picture for a project, or simply to reduce in-house design costs. This user guide will step through the important functions available to the user to simplify their workflow.

# Correlating soil properties from other soil properties:
## Fig_7 (Figure 7 in the NAVFAC Soil Mechanics Manual)

Figure 7 is typically used to estimate the effective friction angle, phi, of a granular soil from the dry unit weight or relative density as a substitute for running expensive lab tests.


Create an instance of Fig_7 and pass in the parameter you want to use to estimate phi and the soil type.
The soil type must be in ['ML','SM','SP','SW','GP','GW']

Estimate phi using Fig_7.read_phi(). This sets the object's phi parameter to the estimated value, and returns the result.

You can also use this figure to estimate relative density from unit weight and vice versa using Fig_7.get_gamma_from_relative_density() and Fig_7.get_relative_density_from_gamma().

Functionality for porosity and void ratio has not been implemented yet because it is not as useful in most cases.

In [ ]:
from geopy import Fig_7
fig = Fig_7(relative_density=75, soil_type='SW')
print(fig.read_phi())
print(fig.get_gamma_from_relative_density())


## Fig_3 (Figure 3 in the NAVFAC Soil Mechanics Manual)

Figure 3 is used to estimate the vertical effective stress at a point in a soil profile based on the relative density and SPT blow count.

Create an instance of Fig_3 and pass in the relative density and blow count.

Estimate effective stress using Fig_3.read_effective_stress(). This sets the object's eff_stress parameter to the estimated value and returns the result. This function works by calculating the effective stress on the two relative density lines from the chart that are closest to the actual relative density, and linearly interpolates the results. This function performs poorly when one of the lines estimates a stress beyond 6 ksf. We attempt to account for this error by capping the individual readings to 6 ksf, but in these cases, you might want to forgoe the use of this function.

In [ ]:
from geopy import Fig_3
fig = Fig_3(relative_density=90, blows=30)
print(fig.read_eff_stress())

## Drilled_Shaft_Design (Designing drilled shaft and driven piles using NAVFAC method)

The Drilled_Shaft_Design class implements the NAVFAC procedure for calculating bearing capacity of deep foundations.
Instantiate the class and pass in the required arguments

design = Drilled_Shaft_Design(depth_water_table_feet=None, foundation_type=None, material=None, 
                 layer_thicknesses=None, layer_unit_weights=None, layer_phi_or_cs=None,
                sublayer_thickness=None, ignored_depth_feet=None)
                
Prepare to analyze the bearing capacity by subdividing the soil profile into several equal layers with create_subdivided_soil_profile()

design.create_subdivided_soil_profile()
                
Analyze several piles simultaneously with the analyze_range method.

design.analyze_range(diameter_list, embedment_list, is_compressive, factor_of_safety)

Typical factor of safety in compression is 2, and in tension is 3.

In [ ]:
from geopy import Drilled_Shaft_Design
x = Drilled_Shaft_Design(depth_water_table_feet=15, foundation_type='drilled pier', material='concrete', 
                         layer_thicknesses=[7,3,8,10,10,5,5,3], 
                         layer_unit_weights=[120,107,115,125,115,117,133,117], 
                         layer_phi_or_cs=[3500,32,30,1400,30,33,1750,32],
                        sublayer_thickness=1, ignored_depth_feet=3)
x.create_subdivided_soil_profile()
x.analyze_range([2,4,6,8],[6,10,20,30,40,50],True,2)
x.analyze_range([2,4,6,8],[6,10,20,30,40,50],False,3)

# Settlement Analysis